In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SC_Dams_Dis_Tracts.csv to SC_Dams_Dis_Tracts (1).csv


In [ ]:
import pandas as pd
import requests
import time

In [ ]:
df = pd.read_csv("SC_Dams_Dis_Tracts.csv")


In [ ]:
# Add columns to store the state, county, and tract codes (excluding block codes)
df['State_FIPS_2010'] = None
df['County_FIPS_2010'] = None
df['Tract_Code_2010'] = None

In [ ]:
# Function to get Census codes based on latitude and longitude
def get_census_codes(lat, lon):
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Census2010_Current&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            # Extract the full GEOID for the block (used here to get state, county, and tract)
            geoid = data['result']['geographies']['Census Blocks'][0]['GEOID']
            # Extract state, county, and tract codes from the GEOID
            state_fips = geoid[:2]
            county_fips = geoid[2:5]
            tract_code = geoid[5:11]
            return state_fips, county_fips, tract_code
        except (KeyError, IndexError, ValueError):
            return None, None, None
    else:
        print(f"Failed to retrieve data for {lat}, {lon} - Status Code: {response.status_code}")
        return None, None, None

In [ ]:
# Apply the function to each row to get state, county, and tract codes
for index, row in df.iterrows():
    state_fips, county_fips, tract_code = get_census_codes(row['latitude'], row['longitude'])
    df.at[index, 'State_FIPS_2010'] = state_fips
    df.at[index, 'County_FIPS_2010'] = county_fips
    df.at[index, 'Tract_Code_2010'] = tract_code
    time.sleep(0.2)  # Add a delay of 200ms


In [ ]:
df2 = df

In [ ]:
df.to_csv("output_with_2010_tracts.csv", index=False)

In [ ]:
from google.colab import files
files.download("output_with_2010_tracts.csv")

In [ ]:
# Add 'Z' to County_FIPS and Tract_Code columns
df2['County_FIPS_2010'] = 'Z' + df2['County_FIPS_2010'].astype(str).str.zfill(3)  # Ensure leading zeros first
df2['Tract_Code_2010'] = 'Z' + df2['Tract_Code_2010'].astype(str).str.zfill(6)  # Ensure leading zeros first

In [ ]:
df2.to_csv("output_with_leading_zeros.csv", index=False)


In [ ]:
from google.colab import files
files.download("output_with_leading_zeros.csv")